In [4]:
!cd .. && pip install -e .

Obtaining file:///Users/ydchen/MBDS_FYP/FYP_v3
  Preparing metadata (setup.py) ... done
  Attempting uninstall: myproject
    Found existing installation: myproject 1.0
    Uninstalling myproject-1.0:
      Successfully uninstalled myproject-1.0
  Running setup.py develop for myproject


In [5]:
import warnings
warnings.filterwarnings(f"ignore")

import pandas as pd
from tqdm import tqdm
import networkx as nx
import sys
import pandas as pd
import json
from collections import defaultdict
import os
import shared
shared.init()

import shared
shared.init()

In [6]:
def distribution_metric(balances, total_supply):
    g1 = sum([(value/total_supply)**2 for holder,value in balances.items() if holder not in [shared.ETH_ADDRESS,
                                                                                             shared.DEAD_ADDRESS]])
    return g1

def get_curve(transfers, type_):
    balances = defaultdict(lambda: 0)
    from_, to_, values, timestamp = 1, 2, 3, 4
    total_supply, total_supply_ans = 0, 0
    i = 0
    while i < len(transfers):
        balances[transfers[i][from_]] -= float(transfers[i][values])
        balances[transfers[i][to_]] += float(transfers[i][values])
        if transfers[i][from_] == shared.ETH_ADDRESS:
            total_supply += float(transfers[i][values])
            balances[transfers[i][from_]] = 0
        if transfers[i][to_] == shared.ETH_ADDRESS:
            total_supply -= float(transfers[i][values])
            balances[transfers[i][to_]] = 0
        i += 1
    if total_supply != 0:
        curve = distribution_metric(balances, total_supply)
    else:
        curve = 1

    return {f'{type_}_curve': curve}


def get_transfer_features(hour_transfers, total_supply):
    num_transactions_list = len(hour_transfers)
    n_unique_addresses = len(set(hour_transfers['from'].tolist() + hour_transfers['to'].tolist()))
    avg_transaction_value = sum(hour_transfers['value'].astype(float))/num_transactions_list
    G = nx.Graph()
    for From, To, Value in \
            zip(hour_transfers['from'].to_numpy(), hour_transfers['to'].to_numpy(), hour_transfers['value'].to_numpy()):
        G.add_edge(From, To, weight=Value)

    try:
        avg_cluster_coeff = nx.average_clustering(G)
    except:
        avg_cluster_coeff = 0

    features = {
        'num_transactions': num_transactions_list,
        'n_unique_addresses': n_unique_addresses,
        'avg_cluster_coeff': avg_cluster_coeff,
        'avg_transaction_value': avg_transaction_value/total_supply
    }
    return features

In [7]:
# 

In [79]:
uniswap_all = list(set(pd.read_csv("../data/uniswap_v2/polygon_pools.csv")['token0'].tolist()+pd.read_csv("../data/uniswap_v2/polygon_pools.csv")['token1'].tolist()))
uniswap_malicious = pd.read_csv("../data/uniswap_v2/polygon_malicious_tokens.csv").T.values.tolist()[0]

In [80]:
# set(list(uniswap_all + uniswap_malicious))

In [81]:
uniswap_malicious

['0xA07e9Df78648E5704275263403A3dC11f159b655',
 '0xBb820a01Bb18f0583969C6733899400aEdFa1E44',
 '0xeD5eDF7E3EDED7b8F679C41010716BbeFA2056d4',
 '0x3385a664c1A24181d89A8498806f7a374073632E',
 '0xE3aeA3dF5d1219dF88Ea455a3C550Eb7056d3fE9',
 '0xDCa1397891F3D245094a20506bFBaF86610A392B',
 '0xC4C341148CF121F1fa962Fca211B0057E0eB7D41',
 '0x669902e2e3cfDD251d8F1D42df1418E3F9f888d1',
 '0x027FCBA0Bf12853269C625C762dba8D59e52cd9c',
 '0xe642F437B2e6D9D69d95a69e9e47CBF76A38E854',
 '0x8DcB537cb2f7563372E4ce253F4c5F0804D3cdc8',
 '0xCAbA280C96ca6F3c64c620C304E77c48Ce473Fda',
 '0x24eb627569DE2a543bd2e87539314abaE694ED9d',
 '0xa608686B8e06B04C2FddF3314349a101CA3441B3',
 '0xDDEA2A076b77071Bd30f86644e6bC109DD3c13c7',
 '0xACD351782fB25Da0181C03592927E55177C42790',
 '0x7385034B28dbA5a3aCc916Aab15a0B2a665A6067',
 '0x1044669Ed21Cd9F75527DF4babC4a35A0b7685D5',
 '0x4CDde30DABe200F1BB4e7593a44A0eA044FCb07e',
 '0x655c6be6709ADA882456667a365De77Fa2eB4764',
 '0x7B6Ddddc65201Cb731035D135F8b88aB3c8C358e',
 '0xa9B7F1f42

In [82]:
uniswap_label = []
for i in uniswap_all:
    if i in uniswap_malicious:
        uniswap_label.append(1)
    else:
        uniswap_label.append(0)
pd.DataFrame({"token": uniswap_all, "label": uniswap_label}).to_csv("../data/uniswap_v2/labeled_list.csv", index = None)

In [83]:
hours_since_creation = 24
protocol = "uniswap_v2"

pools = pd.read_csv(f"../data/{protocol}/polygon_pools.csv")
pools = pools.loc[(pools.token0 == shared.WMATIC) | (pools.token1 == shared.WMATIC)]

tokens = pd.read_csv(f"../data/{protocol}/labeled_list.csv")
time_elapsed = 60*60*hours_since_creation

total_features = {}
total_transfers = pd.DataFrame(columns=['token', 'address', 'price_rev'])

In [84]:
"0x6EEb61Fed9E1654d9065184E77c187aDC1d9222c" in tokens.token

False

In [85]:
tokens.label.value_counts()

0    914
1    303
Name: label, dtype: int64

In [86]:
for token_info in tqdm(tokens.values):
    token_address, label = token_info[0], token_info[1]
    try: 
        pool = pools.loc[(pools.token0 == token_address) | (pools.token1 == token_address)].values[0]
    except:
        continue
    # print(pools.loc[(pools.token0 == token_address) | (pools.token1 == token_address)].values)
    pool_address, token0, token1, block_number, pool_creation_timestamp = pool[0], pool[1], pool[2], pool[3], pool[4]
    total_features[pool_address] = {'token_address': token_address, 'label': label}

    position = 1 if token1 == shared.WMATIC else 0
    token = token0 if position == 1 else token1
    limit_timestamp = pool_creation_timestamp + time_elapsed
    
    # print(pool[0])
    transfers = pd.read_csv(f"../data/{protocol}/transfer/{token}.csv")
    mints = pd.read_csv(f"../data/{protocol}/mint/{pool_address}.csv")
    burns = pd.read_csv(f"../data/{protocol}/burn/{pool_address}.csv")
    swaps = pd.read_csv(f"../data/{protocol}/swap/{pool_address}.csv")

    transfers = transfers.loc[transfers.timestamps < limit_timestamp]
    mints = mints.loc[mints.timestamps < limit_timestamp]
    burns = burns.loc[burns.timestamps < limit_timestamp]
    swaps = swaps.loc[swaps.timestamps < limit_timestamp]

    total_features[pool_address].update({'num_mints': len(mints), 'num_burns': len(burns), 'num_swaps': len(swaps)})

    if 0 in [len(transfers)]:
        continue

    # Pool features
    total_supply = + sum(transfers.loc[(transfers['from'] == shared.ETH_ADDRESS)]['value'].astype(float))\
                   - sum(transfers.loc[(transfers['to'] == shared.ETH_ADDRESS)]['value'].astype(float))
    if total_supply == 0:
        continue

    # Transfer features
    tx_curve = get_curve(transfers.values, type_='tx')
    total_features[pool_address].update(tx_curve)
    transfer_features = get_transfer_features(transfers, total_supply)
    total_features[pool_address].update(transfer_features)
    total_features[pool_address].update({'difference_token_pool': + pool_creation_timestamp
                                                                  - transfers.iloc[0]['timestamps']})

pd.DataFrame(total_features).transpose().to_csv(f"../data/{protocol}/features{hours_since_creation}h.csv", index_label="pool_address")

100%|██████████| 1217/1217 [00:08<00:00, 141.00it/s]


In [30]:
pools

,pair,token0,token1,block_number,timestamp
1,0x531eB8f8E8e626cCFaB2BAA38d14A45096a50180,0x0d500B1d8E8eF31E21C99d1Db9A6444d3ADf1270,0xae20563403DDd9107eC5C7f2AbBc9864E2039Dc8,22766764,1640041518
5,0xfA69bF4fA125ef9Cc74945EAc914150e5b3cF3E0,0x0d500B1d8E8eF31E21C99d1Db9A6444d3ADf1270,0xDF44cb429a57e51152AA4EF99B0119756C721FBF,22773434,1640056593
6,0xb874c2403391fc1571f778fb6fa9A909eb0E79b7,0x0d500B1d8E8eF31E21C99d1Db9A6444d3ADf1270,0xB8060D8fc453561Cfb1d30A6cA36FD3c7669472D,22778803,1640068358
7,0x4f414E6489fD19bBb50C1Bc6C403E19D0dC71Fd2,0x0d500B1d8E8eF31E21C99d1Db9A6444d3ADf1270,0xBf0eA60F6dbB35f9BfE9EA5b532B7795772Af541,22783327,1640078514
16,0x36B860b863C23a1564aD19F28FD4260443E007a2,0x0d500B1d8E8eF31E21C99d1Db9A6444d3ADf1270,0x5BCda6E59262A96a599Ea938c9B679714c105Bba,22802607,1640120002
...,...,...,...,...,...
1440,0x699411dB09112B0b1428C0934C3910797aC7B6A2,0x0d500B1d8E8eF31E21C99d1Db9A6444d3ADf1270,0x32Cd1BCB75473845b5d1dB6ecE60AEC6E41d8518,27400675,1650542516
1441,0x00db81bD9f3bd8378211A68EBD4bA42E9816662a,0x0d500B1d8E8eF31E21C99d1Db9A6444d3ADf1270,0xc43146012A1b2Bf45D992D0fb7bE44667A937fE2,27407741,1650557687
1442,0xB687c2c5DF06Fdc0D93F77163F973fD9060aAc98,0x0d500B1d8E8eF31E21C99d1Db9A6444d3ADf1270,0xCcae792551205B18adBE3B3EBA01a2878BFa6998,27423406,1650590676
1454,0xE0564A53fBc94Cf909e3f258491929B4D31De6Bf,0x08DEEb0AB4d970B50aEF456cBFee1C830FaE6FAb,0x0d500B1d8E8eF31E21C99d1Db9A6444d3ADf1270,27471360,1650695414


In [67]:
tokens

,token,label
0,0x6EEb61Fed9E1654d9065184E77c187aDC1d9222c,0
1,0xCfEC09616c44CbafF3B0D60414e041aa7aaBBF15,0
2,0xCa2a18877D6B715d0E53A69845Ec25237a0315F5,0
3,0x7c7DAAF2dB46fEFd067f002a69FD0BE14AeB159f,0
4,0xF21917bC081aFEa1eC687508C374264D9F721477,0
...,...,...
1212,0xF906695AeAF2aEc43f0343165a909B4A30252F6C,0
1213,0xf9c1F70f9bF57FAD5f63c6E1E25C2e895f04c0A6,0
1214,0xB52CFBca305e13c4C6774846F8d42BB1F0eC51d0,0
1215,0x1C931836980b38E28D50E4027F6880B30CC71861,0


In [17]:
pool

array(['0x345E929eC4b771B73e874777cd9526546bEe3bBe',
       '0x0d500B1d8E8eF31E21C99d1Db9A6444d3ADf1270',
       '0xAa9654BECca45B5BDFA5ac646c939C62b527D394', 16944049, 1626488717],
      dtype=object)